# Getting Started with Aegirio

This example is based on the KubeFlow [Financial Time Series with Finance Data](https://github.com/kubeflow/examples/blob/master/financial_time_series/Financial%20Time%20Series%20with%20Finance%20Data.ipynb) example

This solution presents an example of machine learning workflow using financial time series data.

In this solution, you will:

1. Get data for financial markets using:
    * an SQL query
    * a REST API call
    * an uploaded CSV file
2. Preprocess the data
    * Load into a Pandas Data Frame in a usable format
    * Normalize the data
3.  Develope a model
    * Perform exploratory data analysis
    * Test on subset of full data
4. Train Model at Scale
    * Use TensorFlow to build, train and evaluate a number of models



## Thesis
Can we use information from a stock marketin an earlier time zone to gain an advantage for trading in a later time zone.

We will analyze closing prices from these 8 stock indices from Jan 1st, 2010 thru Oct 1, 2015


|Index|Country|Closing Time (EST)|Hours Before S&P Close|
|---|---|---|---|
|[All Ords](https://en.wikipedia.org/wiki/All_Ordinaries)|Australia|0100|15|
|[Nikkei 225](https://en.wikipedia.org/wiki/Nikkei_225)|Japan|0200|14|
|[Hang Seng](https://en.wikipedia.org/wiki/Hang_Seng_Index)|Hong Kong|0400|12|
|[DAX](https://en.wikipedia.org/wiki/DAX)|Germany|1130|4.5|
|[FTSE 100](https://en.wikipedia.org/wiki/FTSE_100_Index)|UK|1130|4.5|
|[NYSE Composite](https://en.wikipedia.org/wiki/NYSE_Composite)|US|1600|0|
|[Dow Jones Industrial Average](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average)|US|1600|0|
|[S&P 500](https://en.wikipedia.org/wiki/S%26P_500_Index)|US|1600|0|

## 1 - Get the Data
We will be using Pandas data frames to manipulate the data so we import the libraries we will need

In [ ]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
from pandas.plotting import scatter_matrix

### SQL Query

We will get several sets of data from a publicly available BigQuery database

In [ ]:
from google.cloud import bigquery

# Instantiates a client
bigquery_client = bigquery.Client()

These strings are the last part part of the identifier in the BigQuery database. For example `bingo-ml-1.market_data.nyse`

In [ ]:
tickers = ['snp', 'nyse', 'nikkei', 'hangseng', 'ftse', 'dax']

Get raw data

In [ ]:
bq_query = {}
for ticker in tickers:
    bq_query[ticker] = bigquery_client.query('SELECT Date, Close from `bingo-ml-1.market_data.{}`'.format(ticker))

Next we pull the raw data to a temporary `results` object in a Pandas data frame format with the `Date` column as the index

In [ ]:
results = {}
for ticker in tickers:
    results[ticker] = bq_query[ticker].result().to_dataframe().set_index('Date')

We now create our working DataFrame `closing_data` and move just the `Close` column into it

In [ ]:
closing_data = pd.DataFrame()

for ticker in tickers:
    closing_data['{}_close'.format(ticker)] = results[ticker]['Close']

We use the pandas DataFrame `info()` method to see a summary of the data so far

In [ ]:
closing_data.info()

We now have 6 of the 8 indices we want to analyze pulled in our Data Frame using a SQL query


### REST API Call
We will next get the Australian All Ordinaries closing price information from the [Yahoo Finance](https://finance.yahoo.com/quote/%5EAORD/history/) web service

We will need to convert the beginning and end dates of our series to a unix epoch before making the call to the web service

In [ ]:
import time

start_date_time = '01/01/2010'
end_date_time = '10/1/2015'
pattern = '%m/%d/%Y'
start_epoch = int(time.mktime(time.strptime(start_date_time, pattern)))
end_epoch = int(time.mktime(time.strptime(end_date_time, pattern)))

start_epoch, end_epoch

We displayed the values just to be sure they look about right

Next we will use these values to create a correctly formatted URL

In [ ]:
import urllib.request

query_string = 'https://query1.finance.yahoo.com\
/v7/finance/download/%5EAORD?period1={0}&period2={1}&interval=1d&events=history'.format(start_epoch, end_epoch)

response = urllib.request.urlopen(query_string)

Pandas has many options available for reading CSV files into pandas DataFrames. The line below uses three options:
* `usecols` takes an array of column names to be includes in the DataFrame
* `index_col` takes an integer of the column to be used as the DataFrame index
* `date_parser` lets you include a lambda function to use for parsing one or more columns into a pandas Date format

In [ ]:
results['yahooao'] = pd.read_csv(response, \
                                 usecols=['Date', 'Close'],\
                                 index_col=0, \
                                 date_parser=lambda col: pd.to_datetime(col, utc=True))

In [ ]:
results['yahooao'].head()

We use the pandas `head()` to look at the first few rows to be sure they look correct before moving the results into our working DataFrame and adding the name to our `tickers` list

In [ ]:
closing_data['yahooao_close'] = results['yahooao']
tickers.append('yahooao')

In [ ]:
closing_data.info()

The `info()` method now shows the All Ordinaries closing prices we pulled from Yahoo 

### Import CSV file

We will use the same options with pandas `read_csv()` method to import a CSV file. Instead of reading an HTTP response, we will read from a local file `DJIAHistoricalPrices.csv`

In [ ]:
results['djia-csv'] = pd.read_csv('~/DJIAHistoricalPrices.csv', \
                                 usecols=['Date', 'Close'],\
                                 index_col=0, \
                                 date_parser=lambda col: pd.to_datetime(col, utc=True))

You should see an error `ValueError: Usecols do not match columns, columns expected but not found: ['Close']`


Lets read the file in without any parsing and output the columns

In [ ]:
pd.read_csv('~/DJIAHistoricalPrices.csv').columns

Now we see the problem, the 'CLose' column has a leading space

Adding a space in the `usecols` parameter array should fix the problem

In [ ]:
results['djia-csv_close'] = pd.read_csv('~/DJIAHistoricalPrices.csv', \
                                 usecols=['Date', ' Close'],\
                                 index_col=0, \
                                 date_parser=lambda col: pd.to_datetime(col, utc=True))

In [ ]:
results['djia-csv_close']


In [ ]:
closing_data['djia-csv_close'] = results['djia-csv_close']
tickers.append('djia-csv')

In [ ]:
closing_data.info()

We now have our 8 sets of data in a pandas DataFrame, but we can see that due differing national holidays, there are different numbers of data points in each set 

In [ ]:
closing_data.head()

## 2 - Preprocess Data

In this section we will get the data ready by:
* Sorting the closing price values by date and fill in any gaps by repeating the previous days closing price
* Normalize the indicies to have values ranging between 0 and 1

### Sort and Fill missing valures

In [ ]:
# Pandas includes a very convenient function for filling gaps in the data.
closing_data.sort_index(inplace=True)
closing_data = closing_data.fillna(method='ffill')

In [ ]:
closing_data.info()

In [ ]:
closing_data.describe()


At this point we have:
* Sourced five years of financial index time series from a SQL database
* Pulled time series data for specific dates from a Web service
* Imported a CSV file with a mis-named column
* Combined the pertinent data into a single data structure, and harmonized the data to have the same number of entries

### Normalize Data

First, take a look at the data.

In [ ]:
closing_data.describe()

You can see that the various indices operate on scales differing by orders of magnitude. It's best to scale the data so that, for example, operations involving multiple indices aren't unduly influenced by a single, massive index.

Plot the data.

In [ ]:
pd.concat([closing_data['{}_close'.format(ticker)] for ticker in tickers], axis=1).plot(figsize=(20, 15))

As expected, the structure isn't uniformly visible for the indices. Divide each value in an individual index by the maximum value for that index., and then replot. The maximum value of all indices will be 1.

In [ ]:
for ticker in tickers:
    closing_data['{}_close_scaled'.format(ticker)] = closing_data['{}_close'.format(ticker)]/max(closing_data['{}_close'.format(ticker)])

In [ ]:
_ = pd.concat([closing_data['{}_close_scaled'.format(ticker)] for ticker in tickers], axis=1).plot(figsize=(20, 15))

You can see that, over the five-year period, these indices are correlated. Notice that sudden drops from economic events happened globally to all indices, and they otherwise exhibited general rises. This is an good start, though not the complete story. Next, plot autocorrelations for each of the indices. The autocorrelations determine correlations between current values of the index and lagged values of the same index. The goal is to determine whether the lagged values are reliable indicators of the current values. If they are, then we've identified a correlation.

In [ ]:
fig = plt.figure()
fig.set_figwidth(20)
fig.set_figheight(15)

for ticker in tickers:
    _ = autocorrelation_plot(closing_data['{}_close'.format(ticker)], label='{}_close'.format(ticker))

_ = plt.legend(loc='upper right')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


You should see strong autocorrelations, positive for around 500 lagged days, then going negative. This tells us something we should intuitively know: if an index is rising it tends to carry on rising, and vice-versa. It should be encouraging that what we see here conforms to what we know about financial markets.

Next, look at a scatter matrix, showing everything plotted against everything, to see how indices are correlated with each other.

In [ ]:
_ = scatter_matrix(pd.concat([closing_data['{}_close_scaled'.format(ticker)] for ticker in tickers],  axis=1), figsize=(20, 20), diagonal='kde')

You can see significant correlations across the board, further evidence that the premise is workable and one market can be influenced by another.

As an aside, this process of gradual, incremental experimentation and progress is the best approach and what you probably do normally. With a little patience, we'll get to some deeper understanding.

The actual value of an index is not that useful for modeling. It can be a useful indicator, but to get to the heart of the matter, we need a time series that is stationary in the mean, thus having no trend in the data. There are various ways of doing that, but they all essentially look at the difference between values, rather than the absolute value. In the case of market data, the usual practice is to work with logged returns, calculated as the natural logarithm of the index today divided by the index yesterday:
```
ln(Vt/Vt-1)
```
There are more reasons why the log return is preferable to the percent return (for example the log is normally distributed and additive), but they don't matter much for this work. What matters is to get to a stationary time series.

Calculate and plot the log returns in a new DataFrame.

In [ ]:
log_return_data = pd.DataFrame()

for ticker in tickers:
    log_return_data['{}_log_return'.format(ticker)] = np.log(closing_data['{}_close'.format(ticker)]/closing_data['{}_close'.format(ticker)].shift())
    
log_return_data.describe()

Looking at the log returns, you should see that the mean, min, max are all similar. You could go further and center the series on zero, scale them, and normalize the standard deviation, but there's no need to do that at this point. Let's move forward with plotting the data, and iterate if necessary.

In [ ]:
_ = pd.concat([log_return_data['{}_log_return'.format(ticker)] for ticker in tickers], axis=1).plot(figsize=(20, 15))

You can see from the plot that the log returns of our indices are similarly scaled and centered, with no visible trend in the data. It's looking good, so now look at autocorrelations.

In [ ]:
fig = plt.figure()
fig.set_figwidth(20)
fig.set_figheight(15)

for ticker in tickers:
    _ = autocorrelation_plot(log_return_data['{}_log_return'.format(ticker)], label='{}_log_return'.format(ticker))

_ = plt.legend(loc='upper right')